In [ ]:
import numpy as np
import plotly.graph_objects as go

# Define parameters
num_points = 200  # Number of points to generate on the orbit
num_frames = 300  # Number of frames for the animation
animation_duration = 30  # Desired animation duration in seconds

# Calculate the duration of each frame
frame_duration = animation_duration / num_frames

# Generate points on the orbit
theta = np.linspace(0, 2 * np.pi, num_points)
x_orbit = np.cos(theta)
y_orbit = np.sin(theta)
z_orbit = np.zeros(num_points)

# Create the figure
fig = go.Figure()

# Add the black hole event horizon
black_hole_radius = 3.0  # Radius of the black hole
u = np.linspace(0, 2 * np.pi, 100)
v = np.linspace(0, np.pi, 100)
x_horizon = black_hole_radius * np.outer(np.cos(u), np.sin(v))
y_horizon = black_hole_radius * np.outer(np.sin(u), np.sin(v))
z_horizon = black_hole_radius * np.outer(np.ones(np.size(u)), np.cos(v))

fig.add_trace(go.Surface(
    x=x_horizon, y=y_horizon, z=z_horizon,
    colorscale='Greys',
    showscale=False,
    lighting=dict(ambient=0.7, diffuse=0.9),
    opacity=0.5,  # Adjust the opacity of the event horizon
))

# Define the initial position of the particle outside the event horizon
initial_distance = 10.0
x_particle = initial_distance * np.cos(theta)
y_particle = initial_distance * np.sin(theta)
z_particle = np.zeros(num_points)

# Add the falling particle trace
particle = go.Scatter3d(
    x=[x_particle[0]], y=[y_particle[0]], z=[z_particle[0]],
    mode='markers',
    marker=dict(color='blue', size=4),
    name='Particle',
)
fig.add_trace(particle)

# Add the path trace
path = go.Scatter3d(
    x=[x_particle[0]], y=[y_particle[0]], z=[z_particle[0]],
    mode='lines',
    line=dict(color='red', width=2),
    name='Path',
)
fig.add_trace(path)

# Set layout options
fig.update_layout(
    scene=dict(
        xaxis=dict(title='X', range=[-20, 20]),  # Adjust x-axis range
        yaxis=dict(title='Y', range=[-20, 20]),  # Adjust y-axis range
        zaxis=dict(title='Z', range=[-20, 20]),  # Adjust z-axis range
        aspectratio=dict(x=1, y=1, z=1),
        camera=dict(up=dict(x=0, y=0, z=1), eye=dict(x=1.25, y=1.25, z=1.25)),
    ),
    title='Particle Falling into a Black Hole',
    updatemenus=[
        dict(
            type="buttons",
            buttons=[
                dict(
                    label="Play",
                    method="animate",
                    args=[None, {"frame": {"duration": frame_duration * 1000, "redraw": True}, "fromcurrent": True}],
                ),
                dict(
                    label="Pause",
                    method="animate",
                    args=[
                        [None],
                        {"frame": {"duration": 0, "redraw": False}, "mode": "immediate", "transition": {"duration": 0}},
                    ],
                ),
            ],
            direction="left",
            pad={"r": 10, "t": 10},
            showactive=False,
            x=0.1,
            xanchor="right",
            y=0,
            yanchor="top",
        )
    ],
)

# Create animation frames
frames = []
for i in range(num_frames):
    frame = go.Frame(
        data=[
            go.Scatter3d(
                x=[x_particle[i % num_points]],
                y=[y_particle[i % num_points]],
                z=[z_particle[i % num_points]],
                mode='markers',
                marker=dict(color='blue', size=4),
                name='Particle',
            ),
            go.Scatter3d(
                x=x_particle[: i % num_points + 1],
                y=y_particle[: i % num_points + 1],
                z=z_particle[: i % num_points + 1],
                mode='lines',
                line=dict(color='red', width=2),
                name='Path',
            ),
            go.Surface(
                x=x_horizon,
                y=y_horizon,
                z=z_horizon,
                colorscale='Greys',
                showscale=False,
                lighting=dict(ambient=0.7, diffuse=0.9),
                opacity=0.5,  # Adjust the opacity of the event horizon
            ),
        ],
        name=f'frame_{i}',
    )
    frames.append(frame)

# Add frames to the animation
fig.frames = frames

# Create and add slider
steps = []
for i in range(num_frames):
    step = dict(
        method="animate",
        args=[f'frame_{i}'],
        label=f'{i + 1}',
    )
    steps.append(step)

sliders = [
    dict(
        active=0,
        currentvalue={"prefix": "Frame: "},
        pad={"t": 50},
        steps=steps,
    )
]

fig.update_layout(sliders=sliders)

# Show the animation
fig.show()

